In [1]:
with open('/boot/config.txt', 'a') as file_handle:
    file_handle.write('\ndtoverlay=w1-gpio\n')

In [2]:
!cat /sys/bus/w1/devices/28*/w1_slave

96 01 4b 46 7f ff 0a 10 0a : crc=0a YES
96 01 4b 46 7f ff 0a 10 0a t=25375


In [ ]:
import glob
import redislite
from redis_collections import List as RedisList
import time


# Redis RDB backing filename
# By specifying a redisrdb file to redislite we ensure
# the data persists after the program exits.
redisrdb = '/var/lib/redis/temp.rdb'

# How frequently in seconds we want to get the temp readings
frequency = 1
last_reading = 0.0

class DeviceReadError(Exception):
    """
    Exception class for errors reading a device.  This allows
    our program to be able to trap errors reading data from the
    sensor.
    """
    pass


# Functions to read the temp sensor
def read_temp_c():
    """
    Read the temp in Celsius

    Returns
    -------
    temp: float
        The temp from the sensor in Celsius.

    Raises
    ------
    DeviceReadError
        Unable to read values from the temp sesnsor.
    """
    base_dir = '/sys/bus/w1/devices/'
    device_folder = glob.glob(base_dir + '28*')[0]
    device_file = device_folder + '/w1_slave'

    value = -1
    with open(device_file) as file_handle:
        lines = [line.strip().split()[-1] for line in file_handle.readlines()]
    if lines[0] == "YES":
        if lines[1].startswith('t='):
            value = lines[1][2:]
    if value == -1:
        raise DeviceReadError('Unable to access temp sensor')
    return float(value)/1000.0


def read_temp_f():
    """
    Read the temp in Fahrenheit

    Returns
    -------
    temp: float
        The temp from the sensor in Fahrenheit.
    """
    return read_temp_c() * 9.0 / 5.0 + 32.0

In [ ]:
# Redis is key/value store.  It provides a way to store data 
# structures to a service that is seperate from the running 
# program.  Many of the data structures Redis can store are
# nearly identical to standard Python built in data types such
# as lists and dictionaries.  
# We are using the RedisList object from the redis_collections 
# module to access Redis.  This provides us with a python 
# object that works almost exactly like a normal Python list, 
# except the information is stored in Redis.
# This means the information in the list can persist between
# program runs or be stored to mulitple Redis servers using
# Redis repliataion.
# Unlike SQL databases, the default Redis configuration will
# do a single disk write operation every 15 minutes instead
# every time data is written.  This significantly decreases
# wear on flash/sd-card.
# We are going to create two lists in Redis to store our temps
# one for celsius and one for fahranheit.
redis_connection = redislite.StrictRedis(redisrdb)
temp_c_list = RedisList(redis=redis_connection, key='temp_c')
temp_f_list = RedisList(redis=redis_connection, key='temp_f')

del temp_c_list[:]
del temp_f_list[:]

# Now loop through 
while True:
    temp_c_list.append(read_temp_c())
    temp_f_list.append(read_temp_f())
    print('Temp F:', read_temp_f(), 'Hourly Average Temp F:', sum(temp_f_list[-3600:])/len(temp_f_list[-3600:]))
    time.sleep(frequency)


('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.7866)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.7308)
('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.7122)
('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.7308)
('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.74196)
('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.74940000000001)
('Temp F:', 77.7866, 'Hourly Average Temp F:', 77.75471428571429)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.7587)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.7494)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.74195999999999)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.73587272727272)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.73079999999999)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.7177846153846)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.70662857142857)
('Temp F:', 77.675, 'Hourly Average Temp F:', 77.69695999999999)
('Temp F:', 77.5616, 'Hourly Average Temp F:', 77.68849999999999)
('Temp F:', 77.5616, 'H